# Import required packages

In [3]:
import numpy as np
import pandas as pd
import time as time
import random

# Define model and helper functions

In [37]:
class agent:
    """ a datatype representing an agent
        with a given number of features, traits, neighborhood size, and culture
    """
    
    def __init__(self, culture, NumTraits, N):
        """ the constructor for objects of type agent """
        self.culture = culture
        self.numFeatures = len(culture)
        self.numTraits = NumTraits
        self.N = N ## oringinal: neighborhood size --> now: no. of acquaintances 
        self.interactions = 0 
        self.influence = 0  
        self.interaction_times=[] 
        self.interaction_agents=[]
        self.is_city_agent = False ## new attribute
        self.neighbor_count = 0 ## new attribute

    def __repr__(self):
        """ this method returns a string representation
            for an object of type agent
        """
        s=str(self.culture) 
        return s
    
    def reset_neighbor_count(self):
        """Reset the neighbor count for the agent."""
        self.neighbor_count = 0
    
    
class grid:
    """
    a datatype representing a grid
    """
    
    def __init__(self,agentarray, periodic_boundary = False, track_interaction_times= False):
        """ the constructor for objects of type grid """
        self.agentarray = agentarray
        self.rows = len(agentarray)
        self.cols = 1 ## make the grid from 2D to 1D
        self.interaction_count=0
        self.periodic_boundary = periodic_boundary
        
        #Assign a list of neighbors to each agent
        for row in range(self.rows):
            for col in range(self.cols):         
                self.agentarray[row][col].neighbs=listAcquaints(self.rows,self.cols,row,col,agentarray[row][col].N)            
                self.agentarray[row][col].rowcol=(row,col) 
                
        for row in range(self.rows):
            for col in range(self.cols):
                agent = self.agentarray[row][col]
                print(f"Agent {agent.rowcol} initial acquaintances: {agent.neighbs}")
        

    def __repr__(self):
        """ this method returns a string representation
            for an object of type grid
        """
        return str(self.agentarray)
    
    
    ## modified function
    def find_neighb(self,activecoordinates,r1,r2):
        """
        choose an agent from acquaintances or strangers as its neighbor(not geometric neighbor) to interact, 
        and return this agent's coordinates
        """
        active_agent = self.agentarray[activecoordinates[0]][activecoordinates[1]]
        all_agents = []
        for row in range(self.rows):
            for col in range(self.cols):
                all_agents += [[row,col]]
        
        all_agents.remove([activecoordinates[0], activecoordinates[1]]) # Remove the active agent itself
        acquaintances = active_agent.neighbs ## this returns acq's coord.
        stranger_agents = [agent for agent in all_agents if agent not in acquaintances]
        
        result = np.random.choice([0, 1], p=[r1, 1-r1]) ## 0 -> acquaintance; 1 -> stranger   
        #print("acq or stranger", result)
        if result == 0:        
            same_region_acquaintances = [acq for acq in acquaintances if self.agentarray[acq[0]][acq[1]].is_city_agent == active_agent.is_city_agent]
            result_region = np.random.choice([1, 2], p=[r2, 1-r2]) ## 1 -> same_region; 2 -> different_region
            print("acqqqq")
            #print("same or diff region", result_region)
            neighbor_coordinates = (-1,0) #
            print("when neighbor coor still equal to active agent: ", neighbor_coordinates)
            if result_region == 1 and len(same_region_acquaintances) > 0:
                print("same region")
                neighbor_index = np.random.randint(len(same_region_acquaintances))  
                neighbor_coordinates = same_region_acquaintances[neighbor_index] # 
                #print("same region acq", same_region_acquaintances[neighbor_index])
                #return same_region_acquaintances[neighbor_index]
                
            elif result_region == 2 and len(acquaintances) > 0:
                print("diff region")
                neighbor_index = np.random.randint(len(acquaintances))
                neighbor_coordinates = acquaintances[neighbor_index] #
                #print("acq", acquaintances[neighbor_index])
                #return acquaintances[neighbor_index]
            
            ##
            # Update the count for the chosen neighbor
            if neighbor_coordinates is not None:
                print("neighbor coordinate",neighbor_coordinates)
                neighbor_agent = self.agentarray[neighbor_coordinates[0]][neighbor_coordinates[1]]
                neighbor_agent.neighbor_count += 1

                # Check if the neighbor has been chosen more than 5 times, remove from the acquaintance list
                #if neighbor_agent.neighbor_count < 10:
                #    active_agent.neighbs.remove(neighbor_coordinates)
            
            return neighbor_coordinates
            ##
                
        elif result == 1:
            print("strangerrr")
            neighbor_index = np.random.randint(len(stranger_agents))
            #print("stranger", tuple(stranger_agents[neighbor_index]))
            return tuple(stranger_agents[neighbor_index])
    
    
    
    def update_neighbor_counts(self):
        """Update the neighbor counts for all agents."""
        for row in range(self.rows):
            for col in range(self.cols):
                self.agentarray[row][col].reset_neighbor_count()

    def remove_inactive_agents(self, threshold=5):
        """Remove inactive agents from acquaintances."""
        print("removeeeee")
        for row in range(self.rows):
            for col in range(self.cols):
                print("remove for row col")
                active_agent = self.agentarray[row][col]
                inactive_neighbors = [neighbor for neighbor in active_agent.neighbs if self.agentarray[neighbor[0]][neighbor[1]].neighbor_count < threshold]
                print("inactive neighbors:", inactive_neighbors)
                for inactive_neighbor in inactive_neighbors:
                    print("inactive neighbor count", self.agentarray[inactive_neighbor[0]][inactive_neighbor[1]].neighbor_count)
                    active_agent.neighbs.remove(inactive_neighbor)
    
    
    def similarity(self,agent1coordinates,agent2coordinates):
        """
        returns percentage similarity between coordiates for two agents
        inputs: agent 1 coordinates (tuple),agent 2 coordinates (tuple)
        """
       #Pick up the active and neighb agent objects
        active=self.agentarray[agent1coordinates[0]][agent1coordinates[1]]
        neighb=self.agentarray[agent2coordinates[0]][agent2coordinates[1]]
        
        #Count similarity
        similarity = 0 
        i=0
        for a in active.culture:
            if neighb.culture[i]==a:
                similarity+=1
            i+=1
        probability = (similarity / active.numFeatures) * 100
        return probability
        
    def interact(self,active_coordinates,neighb_coordinates,probability, track_interaction_times=False):
        """
        facilitates an interaction between given active agent and their neighbor - 'similarity criterion'
        if similarity = 100%, guarenteed interaction [but no change in culture]
        if similarity = 0%, no interaction
        """
        #Pick up the active and neighb agent objects
        active=self.agentarray[active_coordinates[0]][active_coordinates[1]]
        neighb=self.agentarray[neighb_coordinates[0]][neighb_coordinates[1]]
        
        if probability==100 or probability==0: #already the same culture or cant interact; stop here. 
            return
        
        roll=np.random.rand()*100 # float so fractions of similarity work
        if roll>=probability: #no interaction; stop here
            return
        
        #find feature to change that is not already shared
        different_feature_array = active.culture!=neighb.culture 
        different_feature_indices = np.where(different_feature_array==True)[0]
        random_feature_index = np.random.choice(different_feature_indices)
        
        #change the features
        active.culture[random_feature_index]=neighb.culture[random_feature_index] #change active culture feature trait to the neighbors trait
        active.interactions+=1
        neighb.influence+=1
        self.interaction_count+=1
        if track_interaction_times == True:
            active.interaction_times+=[self.interaction_count]
            active.interaction_agents+=[neighb.rowcol]
        
    def count_total_interactions(self):
        """
        sums total agent interactions
        """
        total = 0 
        for row in range(self.rows):
            for col in range(self.cols):
                total += self.agentarray[row][col].interactions
        return total

    def is_stable(self, agentCoordintes):
        """
        returns True of agent is stable (all neighbors are same or 100% different)
        returns False otherwise
        """
        active=self.agentarray[agentCoordintes[0]][agentCoordintes[1]]
        
        for neighb in active.neighbs:
            sim=self.similarity(agentCoordintes,neighb)
            if sim>0 and sim<100: # between zero and 100 exclusive
                return False
        return True
    
    def count_active_bonds(self, agentCoordintes):
        """
        returns number of agent's active bonds
        """
        active=self.agentarray[agentCoordintes[0]][agentCoordintes[1]]
        num_active_bonds = 0
        for neighb in active.neighbs:
            sim=self.similarity(agentCoordintes,neighb)
            if sim>0 and sim<100: # between zero and 100 exclusive
                num_active_bonds +=1
        return num_active_bonds
    
    def count_all_active_bonds(self):
        num_active_bonds = 0
        for row in range(self.rows):
            for col in range(self.cols):
                num_active_bonds += self.count_active_bonds((row,col))
        return num_active_bonds    
    
    
    def is_grid_stable(self):
        for row in range(self.rows):
            for col in range(self.cols):
                if g.is_stable((row,col))==False:
                    return False
        return True
  

    ## modified function
    def pick_active_agent(self, ratio):
        """
        pick an active agent to interact with others
        at "ratio" persentage will pick a city agent
        at "1-ratio" persentage will pick a rural agent
        """
        all_agents = []
        for row in range(self.rows):
            for col in range(self.cols):
                all_agents.append(self.agentarray[row][col])
                               
        city_agents = [agent for agent in all_agents if agent.is_city_agent]
        rural_agents = [agent for agent in all_agents if not agent.is_city_agent]
        result = np.random.choice([0, 1], p=[ratio, 1-ratio])
        if result == 0:
            active_agent = random.sample(city_agents, 1)[0]
        elif result == 1:
            active_agent = random.sample(rural_agents, 1)[0]
            
        for row in range(self.rows):
            for col in range(self.cols):
                if self.agentarray[row][col] == active_agent:
                    return (row, col)
        
        
    def save_csv(self, filename):
        """
        saves the grid to a csv file
        returns the dataframe
        """
        to_save=np.zeros((self.rows,self.cols),dtype='<U40') # make x in Ux greater if your strings dont complete in csv
        for row in range(self.rows):
            for col in range(self.cols):
                to_save[row][col]=str(self.agentarray[row][col])
        print("Saving grid as ", filename,".csv")
        df = pd.DataFrame(to_save)
        df.to_csv(filename+".csv")
        return df
        
    def save_interactions_csv(self, filename):
        """
        saves the grid's interaction count to a csv file
        returns the dataframe
        """
        to_save=np.zeros((self.rows,self.cols)) 
        for row in range(self.rows):
            for col in range(self.cols):
                to_save[row][col]=self.agentarray[row][col].interactions
        print("Saving grid as ", filename,".csv")
        df = pd.DataFrame(to_save)
        df.to_csv(filename+".csv") 
        return df

    def save_influence_csv(self, filename):
        """
        saves the grid's interaction count to a csv file
        returns the dataframe
        """
        to_save=np.zeros((self.rows,self.cols)) 
        for row in range(self.rows):
            for col in range(self.cols):
                to_save[row][col]=self.agentarray[row][col].influence
        print("Saving grid as ", filename,".csv")
        df = pd.DataFrame(to_save)
        df.to_csv(filename+".csv")
        return df

    def save_interaction_times_csv(self, filename):
        """
        saves the grid's interaction timing and agents with which each interaction occurs to a csv file
        returns the dataframe
        """
        to_save=pd.DataFrame()
        
        for row in range(g.rows):
            for col in range(g.cols):
                to_save[g.agentarray[row][col].rowcol]=g.agentarray[row][col].interaction_times
                to_save[str(g.agentarray[row][col].rowcol)+" with neighbor:"]=g.agentarray[row][col].interaction_agents
        to_save.to_csv(filename+".csv")
        print("Saving times as ", filename,".csv")
        return to_save
    
    
    def count_cultures(self):
        """
        creates a dict of each unique culture + num agents w/ each culture
        """
        culturelist=[]
        for row in range(self.rows):
            for col in range(self.cols):
                culturelist+=[str(self.agentarray[row][col].culture)]      
        counts = dict()
        for i in culturelist: #code from stack overflow
          counts[i] = counts.get(i, 0) + 1 #
        return counts
    
    def save_culture_size_csv(self,filename):
        """
        Saves a cvs grid and returns a dataframe where each agent's culture size is in each agent's respective cell
        """
        counts=self.count_cultures()        
        to_save=np.zeros((self.rows,self.cols)) 
        for row in range(self.rows):
            for col in range(self.cols):
                to_save[row][col]=counts[str(self.agentarray[row][col].culture)]
        print("Saving grid as ", filename,".csv")
        df = pd.DataFrame(to_save)
        df.to_csv(filename+".csv")
        return df 
    
    def print_final_acq_lists(self):
        """
        Print the final acquaintance lists for each agent.
        """
        print("print final acq:")
        for row in range(self.rows):
            for col in range(self.cols):
                print("there remain some acq")
                agent_coordinates = (row, col)
                active_agent = self.agentarray[row][col]
                print(f"Agent {agent_coordinates} final acq list: {active_agent.neighbs}")

## modified function
def listAcquaints(num_rows, num_cols, agent_row, agent_col, num_acquaintances):
    """
    generates a random list of given agent's acquaintances
    inputs: num_rows, num_columns, the agent's row, the agent's column, 
            the number of acquaintances
    outputs: a list contains the tuples of the coordinates of all acquaintances' 
    """
    all_agents = []
    for row in range(num_rows):
        for col in range(num_cols):
            all_agents += [[row,col]]
    all_agents.remove([agent_row, agent_col])  # Remove the agent itself  
    acquaintances = random.sample(all_agents, num_acquaintances)
    return [tuple(x) for x in acquaintances]


# for counting regions 
def assign_unique_cultures(grid):
    """
    helper function for create_community_grid. 
    input ACM grid
    assigns set variable culture_assignment to each agent. Sets contains each agents coordinates
    """
    for row in range(grid.rows):
        for col in range(grid.cols):
            grid.agentarray[row][col].culture_assignment = {(row,col)} # a set with a tuple inside
            

def create_community_grid(grid):
    """
    input an ACM grid
    modifies agents in the grid such that each stores a list culture_assignment of all coordinates of agents in the cultural region  
    """
    assign_unique_cultures(grid) #set unique culture_assignment to each agent in the grid
    for row in range(grid.rows):
        for col in range(grid.cols):
            a = grid.agentarray[row][col]
            a_culture = a.culture
            for n_coordinates in a.neighbs: #Go through every neighbor
                n = grid.agentarray[n_coordinates[0]][n_coordinates[1]]
                n_culture = n.culture
                if np.all(a_culture == n_culture): #If the neighbor has the same culture
                    a.culture_assignment.update(n.culture_assignment)  #merge the two communities
                    for culture_member_coordinates in a.culture_assignment:
                        cm= grid.agentarray[culture_member_coordinates[0]][culture_member_coordinates[1]]
                        cm.culture_assignment = a.culture_assignment
                                        

def count_cultural_regions(grid, col_min, col_max, remove_largest_culture = True):
    """
    NOTE: input must be operated on by create_community_grid for this function to work
    input: ACM grid
    returns: The number of cultural regions in the given column range (inclusive)
    """
    max_size = 0
    max_rowcol = (-1, -1)
    arr = np.zeros((grid.rows,grid.cols))
    
    for row in range(grid.rows):
        for col in range(grid.cols):  
            a = grid.agentarray[row][col] #agent object
            region_size = len(a.culture_assignment)
            arr[row,col] = 1 / region_size
            if region_size > max_size:
                max_size = region_size
                max_rowcol = (row,col)
            
    if remove_largest_culture == True:
        a_max = grid.agentarray[max_rowcol[0]][max_rowcol[1]] #this agent is a memeber of the largest culture
        for agent_coordinates in a_max.culture_assignment:
            arr[agent_coordinates[0],agent_coordinates[1]] = 0
            
    return arr[:,col_min:col_max+1].sum()



def setup_rand_grid(rows,cols,features,traits,N, periodic_boundary=False, track_interaction_times=False):
    """
    creates a grid object with random trait values in each feture slot
    """
    _grid_=np.zeros((rows,cols),dtype=object)
    for row in range(rows):
        for col in range(cols):
            a=agent(np.random.randint(traits, size=features),traits,N)
            _grid_[row,col]=a
    return grid(_grid_, periodic_boundary=periodic_boundary,track_interaction_times=track_interaction_times) # create grid object


## modified function
def assign_city_agents(grid, num_city_agents):
    """
    randomly assign some agents as city agents
    return a list of tuples of their coordinates
    """
    num_total_agents = len(grid.agentarray)
    all_agents = []
    for row in range(grid.rows):
        for col in range(grid.cols):
            all_agents += [[row,col]]
    
    city_indices = random.sample(range(num_total_agents), num_city_agents)
    city_agents = []
    for idx in city_indices:
        grid.agentarray[idx][0].is_city_agent = True
        city_agents.append((idx,0))
    
    print("city agents: ", city_agents)
    


# Set model parameters

In [5]:
#_________________Free Parameters_________________
rows=15 ## total number of agents
cols = 1
features=5 #F
traits=15 #q
N=5 ## number of acquaintances of each agent
city_N=7 ##number of city agents
active_agent_ratio = 0.75
neighb_acq_ratio = 0.75
neighb_same_region_ratio = 0.75

periodic_boundary = False

#_________________End Free Parameters_________________


track_interaction_times = False

trial_name = str(rows) + "x" + "1" + "_" + str(features) + "f_" + str(traits) + "t_" + str(active_agent_ratio) + "_N" + str(city_N)

if periodic_boundary:
    trial_name = trial_name + "_periodic"
print(trial_name)

g=setup_rand_grid(rows,cols,features,traits,N, periodic_boundary=periodic_boundary, track_interaction_times=track_interaction_times)
assign_city_agents(g,city_N)

15x1_5f_15t_0.75_N7
Agent (0, 0) initial acquaintances: [(6, 0), (1, 0), (7, 0), (11, 0), (5, 0)]
Agent (1, 0) initial acquaintances: [(12, 0), (4, 0), (13, 0), (3, 0), (14, 0)]
Agent (2, 0) initial acquaintances: [(9, 0), (8, 0), (12, 0), (7, 0), (6, 0)]
Agent (3, 0) initial acquaintances: [(6, 0), (5, 0), (9, 0), (2, 0), (14, 0)]
Agent (4, 0) initial acquaintances: [(6, 0), (12, 0), (14, 0), (13, 0), (0, 0)]
Agent (5, 0) initial acquaintances: [(3, 0), (4, 0), (11, 0), (9, 0), (14, 0)]
Agent (6, 0) initial acquaintances: [(7, 0), (5, 0), (10, 0), (2, 0), (1, 0)]
Agent (7, 0) initial acquaintances: [(11, 0), (14, 0), (6, 0), (9, 0), (0, 0)]
Agent (8, 0) initial acquaintances: [(4, 0), (12, 0), (2, 0), (14, 0), (10, 0)]
Agent (9, 0) initial acquaintances: [(8, 0), (12, 0), (11, 0), (13, 0), (7, 0)]
Agent (10, 0) initial acquaintances: [(2, 0), (4, 0), (11, 0), (0, 0), (12, 0)]
Agent (11, 0) initial acquaintances: [(13, 0), (5, 0), (3, 0), (14, 0), (0, 0)]
Agent (12, 0) initial acquaint

# Run the model 

In [40]:
num_trials=5

#Create dataframes for collecting averages
df_interactions = pd.DataFrame(np.zeros((rows,cols)))
df_influence = pd.DataFrame(np.zeros((rows,cols)))
df_sizes = pd.DataFrame(np.zeros((rows,cols)))
df_active_bond_density=pd.DataFrame(columns=["Number of Ticks", "Number of Interactions", "Number of Active Bonds"])
if track_interaction_times == True:
    df_interaction_times = g.save_interaction_times_csv(trial_name+"_times_")

#Create dataframe for collecting data across trials
df=pd.DataFrame(columns=['Simulation', 'Graph Name', 'Total Regions', 'Left', 'Middle','Right'])

for trial in range(num_trials):
    time1=time.time()
    g=setup_rand_grid(rows,cols,features,traits,N,periodic_boundary=periodic_boundary)
    assign_city_agents(g,city_N)
    ticks=0
    interactions = 0
    density_list = []
    while True:
        active = g.pick_active_agent(active_agent_ratio)
        print("active in main", active)
        if len(g.agentarray[active[0]][active[1]].neighbs) != 0:
            print("neighb length", len(g.agentarray[active[0]][active[1]].neighbs))
            print("neighbs:", g.agentarray[active[0]][active[1]].neighbs)
            neighb = g.find_neighb(active,neighb_acq_ratio,neighb_same_region_ratio)
        else: continue
        print("neighb in main", neighb)
        probability= g.similarity(active,neighb)
        g.interact(active,neighb,probability)
        #g.remove_inactive_agents(active, threshold=10)
        #g.update_neighbor_counts()
        
        if ticks%100 == 0: #Check if stable every 100000 ticks
            df_active_bond_density = df_active_bond_density.append({"Number of Ticks": ticks, "Number of Interactions": g.count_total_interactions(), "Number of Active Bonds":g.count_all_active_bonds(),},ignore_index=True)
            if g.is_grid_stable()== True: #Simulation completed
                #Saving the grid, interactions, influence, and sizes data to csv files. 
                timestamp=time.strftime("%d_%b_%Y_%H-%M-%S", time.gmtime())
                #g.save_csv(trial_name+"_complete_"+timestamp)
                #df_interactions+=g.save_interactions_csv(trial_name+"_interactions_"+timestamp)
                #print(df_interactions)
                #df_influence+=g.save_influence_csv(trial_name+"_influence_"+timestamp)
                #df_sizes+=g.save_culture_size_csv(trial_name+"_sizes_"+timestamp)
                total_interactions = g.count_total_interactions
                #if track_interaction_times == True:  
                #    df_interaction_times+=g.save_interaction_times_csv(trial_name+"_times_"+timestamp)

                #for region counting: 
                create_community_grid(g)
                total_count = count_cultural_regions(g,0,rows,remove_largest_culture = True)

                #Save data to a data frame
                #df = df.append({'Simulation': trial_name,'Graph Name':trial_name+"_complete_"+timestamp, 'Total Regions':total_count, "Number of Interactions": g.count_total_interactions()}, ignore_index=True)
                #df.to_csv('Data_'+trial_name+'.csv')
                time2=time.time()
                print("Trial time: ", time2-time1)
                print(df)
                g.remove_inactive_agents(threshold=50)
                #g.print_final_acq_lists()
                break
        ticks+=1
    g.print_final_acq_lists()
    g.update_neighbor_counts()
    

#Take the averages
df_interactions=df_interactions/num_trials
df_influence=df_influence/num_trials
df_sizes=df_sizes/num_trials

#Save the averages
#df_interactions.to_csv('Avg-Interactions_'+trial_name+'.csv')
#df_influence.to_csv('Avg-Influence'+trial_name+'.csv')
#df_sizes.to_csv('Avg-Sizes_'+trial_name+'.csv')

Agent (0, 0) initial acquaintances: [(9, 0), (11, 0), (14, 0), (4, 0), (2, 0)]
Agent (1, 0) initial acquaintances: [(12, 0), (14, 0), (2, 0), (5, 0), (8, 0)]
Agent (2, 0) initial acquaintances: [(14, 0), (3, 0), (7, 0), (10, 0), (9, 0)]
Agent (3, 0) initial acquaintances: [(4, 0), (11, 0), (0, 0), (8, 0), (6, 0)]
Agent (4, 0) initial acquaintances: [(6, 0), (13, 0), (3, 0), (14, 0), (9, 0)]
Agent (5, 0) initial acquaintances: [(10, 0), (1, 0), (2, 0), (14, 0), (4, 0)]
Agent (6, 0) initial acquaintances: [(9, 0), (12, 0), (8, 0), (7, 0), (3, 0)]
Agent (7, 0) initial acquaintances: [(3, 0), (2, 0), (11, 0), (8, 0), (14, 0)]
Agent (8, 0) initial acquaintances: [(3, 0), (1, 0), (5, 0), (0, 0), (9, 0)]
Agent (9, 0) initial acquaintances: [(0, 0), (10, 0), (11, 0), (2, 0), (12, 0)]
Agent (10, 0) initial acquaintances: [(14, 0), (12, 0), (8, 0), (0, 0), (6, 0)]
Agent (11, 0) initial acquaintances: [(13, 0), (2, 0), (0, 0), (12, 0), (1, 0)]
Agent (12, 0) initial acquaintances: [(13, 0), (9, 0)

C:\Users\user\AppData\Local\Temp\ipykernel_14776\3821741505.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_active_bond_density = df_active_bond_density.append({"Number of Ticks": ticks, "Number of Interactions": g.count_total_interactions(), "Number of Active Bonds":g.count_all_active_bonds(),},ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14776\3821741505.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_active_bond_density = df_active_bond_density.append({"Number of Ticks": ticks, "Number of Interactions": g.count_total_interactions(), "Number of Active Bonds":g.count_all_active_bonds(),},ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14776\3821741505.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

strangerrr
neighb in main (12, 0)
active in main (10, 0)
neighb length 5
neighbs: [(14, 0), (12, 0), (8, 0), (0, 0), (6, 0)]
acqqqq
when neighbor coor still equal to active agent:  (-1, 0)
same region
neighbor coordinate (14, 0)
neighb in main (14, 0)
active in main (3, 0)
neighb length 5
neighbs: [(4, 0), (11, 0), (0, 0), (8, 0), (6, 0)]
acqqqq
when neighbor coor still equal to active agent:  (-1, 0)
same region
neighbor coordinate (4, 0)
neighb in main (4, 0)
active in main (1, 0)
neighb length 5
neighbs: [(12, 0), (14, 0), (2, 0), (5, 0), (8, 0)]
strangerrr
neighb in main (7, 0)
active in main (14, 0)
neighb length 5
neighbs: [(9, 0), (5, 0), (8, 0), (7, 0), (13, 0)]
strangerrr
neighb in main (10, 0)
active in main (6, 0)
neighb length 5
neighbs: [(9, 0), (12, 0), (8, 0), (7, 0), (3, 0)]
acqqqq
when neighbor coor still equal to active agent:  (-1, 0)
same region
neighbor coordinate (9, 0)
neighb in main (9, 0)
active in main (9, 0)
neighb length 5
neighbs: [(0, 0), (10, 0), (11, 0),

C:\Users\user\AppData\Local\Temp\ipykernel_14776\3821741505.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_active_bond_density = df_active_bond_density.append({"Number of Ticks": ticks, "Number of Interactions": g.count_total_interactions(), "Number of Active Bonds":g.count_all_active_bonds(),},ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14776\3821741505.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_active_bond_density = df_active_bond_density.append({"Number of Ticks": ticks, "Number of Interactions": g.count_total_interactions(), "Number of Active Bonds":g.count_all_active_bonds(),},ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14776\3821741505.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

acqqqq
when neighbor coor still equal to active agent:  (-1, 0)
diff region
neighbor coordinate (3, 0)
neighb in main (3, 0)
active in main (8, 0)
neighb length 5
neighbs: [(3, 0), (1, 0), (5, 0), (0, 0), (9, 0)]
strangerrr
neighb in main (9, 0)
active in main (3, 0)
neighb length 5
neighbs: [(4, 0), (11, 0), (0, 0), (8, 0), (6, 0)]
strangerrr
neighb in main (10, 0)
active in main (10, 0)
neighb length 5
neighbs: [(14, 0), (12, 0), (8, 0), (0, 0), (6, 0)]
acqqqq
when neighbor coor still equal to active agent:  (-1, 0)
diff region
neighbor coordinate (14, 0)
neighb in main (14, 0)
active in main (5, 0)
neighb length 5
neighbs: [(10, 0), (1, 0), (2, 0), (14, 0), (4, 0)]
strangerrr
neighb in main (7, 0)
active in main (7, 0)
neighb length 5
neighbs: [(3, 0), (2, 0), (11, 0), (8, 0), (14, 0)]
acqqqq
when neighbor coor still equal to active agent:  (-1, 0)
diff region
neighbor coordinate (8, 0)
neighb in main (8, 0)
active in main (0, 0)
neighb length 5
neighbs: [(9, 0), (11, 0), (14, 0), (

...

C:\Users\user\AppData\Local\Temp\ipykernel_14776\3821741505.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_active_bond_density = df_active_bond_density.append({"Number of Ticks": ticks, "Number of Interactions": g.count_total_interactions(), "Number of Active Bonds":g.count_all_active_bonds(),},ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14776\3821741505.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_active_bond_density = df_active_bond_density.append({"Number of Ticks": ticks, "Number of Interactions": g.count_total_interactions(), "Number of Active Bonds":g.count_all_active_bonds(),},ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14776\3821741505.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

acqqqq
when neighbor coor still equal to active agent:  (-1, 0)
same region
neighbor coordinate (2, 0)
neighb in main (2, 0)
active in main (6, 0)
neighb length 5
neighbs: [(4, 0), (11, 0), (0, 0), (14, 0), (10, 0)]
acqqqq
when neighbor coor still equal to active agent:  (-1, 0)
same region
neighbor coordinate (10, 0)
neighb in main (10, 0)
active in main (13, 0)
neighb length 5
neighbs: [(4, 0), (6, 0), (12, 0), (7, 0), (10, 0)]
acqqqq
when neighbor coor still equal to active agent:  (-1, 0)
same region
neighbor coordinate (7, 0)
neighb in main (7, 0)
active in main (0, 0)
neighb length 5
neighbs: [(8, 0), (1, 0), (3, 0), (10, 0), (11, 0)]
strangerrr
neighb in main (4, 0)
active in main (7, 0)
neighb length 5
neighbs: [(14, 0), (0, 0), (3, 0), (5, 0), (2, 0)]
acqqqq
when neighbor coor still equal to active agent:  (-1, 0)
same region
neighbor coordinate (2, 0)
neighb in main (2, 0)
active in main (5, 0)
neighb length 5
neighbs: [(3, 0), (13, 0), (2, 0), (11, 0), (7, 0)]
acqqqq
when ne

C:\Users\user\AppData\Local\Temp\ipykernel_14776\3821741505.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_active_bond_density = df_active_bond_density.append({"Number of Ticks": ticks, "Number of Interactions": g.count_total_interactions(), "Number of Active Bonds":g.count_all_active_bonds(),},ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14776\3821741505.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_active_bond_density = df_active_bond_density.append({"Number of Ticks": ticks, "Number of Interactions": g.count_total_interactions(), "Number of Active Bonds":g.count_all_active_bonds(),},ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14776\3821741505.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

active in main (1, 0)
neighb length 5
neighbs: [(0, 0), (8, 0), (4, 0), (11, 0), (10, 0)]
acqqqq
when neighbor coor still equal to active agent:  (-1, 0)
diff region
neighbor coordinate (11, 0)
neighb in main (11, 0)
active in main (14, 0)
neighb length 5
neighbs: [(3, 0), (0, 0), (9, 0), (1, 0), (11, 0)]
acqqqq
when neighbor coor still equal to active agent:  (-1, 0)
same region
neighbor coordinate (3, 0)
neighb in main (3, 0)
active in main (2, 0)
neighb length 5
neighbs: [(0, 0), (4, 0), (7, 0), (1, 0), (10, 0)]
acqqqq
when neighbor coor still equal to active agent:  (-1, 0)
same region
neighbor coordinate (0, 0)
neighb in main (0, 0)
active in main (1, 0)
neighb length 5
neighbs: [(0, 0), (8, 0), (4, 0), (11, 0), (10, 0)]
acqqqq
when neighbor coor still equal to active agent:  (-1, 0)
same region
neighbor coordinate (0, 0)
neighb in main (0, 0)
active in main (8, 0)
neighb length 5
neighbs: [(9, 0), (1, 0), (2, 0), (13, 0), (0, 0)]
strangerrr
neighb in main (14, 0)
active in main (

C:\Users\user\AppData\Local\Temp\ipykernel_14776\3821741505.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_active_bond_density = df_active_bond_density.append({"Number of Ticks": ticks, "Number of Interactions": g.count_total_interactions(), "Number of Active Bonds":g.count_all_active_bonds(),},ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14776\3821741505.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_active_bond_density = df_active_bond_density.append({"Number of Ticks": ticks, "Number of Interactions": g.count_total_interactions(), "Number of Active Bonds":g.count_all_active_bonds(),},ignore_index=True)
